In [1]:
# Required Packages
import pandas as pd
from datetime import datetime
import numpy as np

import warnings
warnings.filterwarnings("ignore")

#### Table of Contents

* [Los Angeles County Restaurant and Market Violations and Inspections](#Los-Angeles-County-Restaurant-and-Market-Violations-and-Inspections)
    * [Violations Dataset](#Violations-Dataset)
    * [Inspection Result Dataset](#Inspection-Result-Dataset)
* [Preprocessing](#Preprocessing)
    * [Inspection Dataset](#Inspection-Dataset)
    * [Creating a new Dataset](#Creating-a-new-Dataset)
        * [Reindexing](#Reindexing)
    * [Creating a Dictionary](#Creating-a-Dictionary)

# Los Angeles County Restaurant and Market Violations and Inspections


In this article, we use a dataset contains Environmental Health Violations for Restaurants and Markets in Los Angeles County. This dataset can be found [here](https://data.lacounty.gov/Health/LOS-ANGELES-COUNTY-RESTAURANT-AND-MARKET-VIOLATION/8jyd-4pv9) and [here](https://data.lacounty.gov/Health/LOS-ANGELES-COUNTY-RESTAURANT-AND-MARKET-INSPECTIO/6ni6-h5kp).

Los Angeles County Environmental Health is responsible for checking food violations for all unincorporated areas and 85 of the 88 cities in the County. This dataset does not include Pasadena, Long Beach or Vernon (each has its own city health department).

## Violations Dataset

Each row represents one health code violation. All rows with the same Serial Number are part of the same inspection. The Serial Number is the primary key to review the inspection grade (Inspection Result dataset)

In [2]:
Violations_df = pd.read_csv('Data/LA/LOS_ANGELES_COUNTY_RESTAURANT_AND_MARKET_VIOLATIONS.csv')
Violations_df.head().style.hide_index()

SERIAL NUMBER,VIOLATION STATUS,VIOLATION CODE,VIOLATION DESCRIPTION,POINTS
DA000211Z,OUT OF COMPLIANCE,F006,# 06. Adequate handwashing facilities supplied & accessible,2.000000
DA000211Z,OUT OF COMPLIANCE,F044,"# 44. Floors, walls and ceilings: properly built, maintained in good repair and clean",1.000000
DA000211Z,OUT OF COMPLIANCE,F014,# 14. Food contact surfaces: clean and sanitized,2.000000
DA000211Z,OUT OF COMPLIANCE,F029,"# 29. Toxic substances properly identified, stored, used",1.000000
DA000211Z,OUT OF COMPLIANCE,F035,"# 35. Equipment/Utensils - approved; installed; clean; good repair, capacity",1.000000


## Inspection Result Dataset

Each row represents one inspection result. The Serial Number is the primary key to access/review the violations (Violations Dataset) associated with a particular inspection.

In [3]:
Inspections_df = pd.read_csv('Data/LA/LOS_ANGELES_COUNTY_RESTAURANT_AND_MARKET_INSPECTIONS.csv')
Inspections_df.head().style.hide_index()

ACTIVITY DATE,OWNER ID,OWNER NAME,FACILITY ID,FACILITY NAME,RECORD ID,PROGRAM NAME,PROGRAM STATUS,PROGRAM ELEMENT (PE),PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY STATE,FACILITY ZIP,SERVICE CODE,SERVICE DESCRIPTION,SCORE,GRADE,SERIAL NUMBER,EMPLOYEE ID,Location
06/02/2017,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,1111 W WASHINGTON BLVD STE H,MONTEBELLO,CA,90640,1,ROUTINE INSPECTION,91,A,DAXE7ACTS,EE0000773,nan
06/05/2017,OW0025119,MORNING SUN INC.,FA0008772,CHINA GATE,PR0029024,CHINA GATE,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,11814 S WILMINGTON AVE,LOS ANGELES,CA,90059,1,ROUTINE INSPECTION,94,A,DADMCPXSY,EE0000927,nan
06/07/2017,OW0007353,"CHEN, CHAO SHU",FA0070050,SAN HING,PR0004325,SAN HING,INACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,11136 PALMS BLVD,LOS ANGELES,CA,90034,1,ROUTINE INSPECTION,96,A,DAJMZ6U86,EE0000911,POINT (-118.422231 34.01925)
06/12/2017,OW0100197,"SBARRO, LLC",FA0032188,SBARRO,PR0018712,SBARRO,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,3650 W MLK BLVD # FC-10,LOS ANGELES,CA,90008,1,ROUTINE INSPECTION,90,A,DAMPB9559,EE0000176,nan
06/15/2017,OW0028751,PO SOKHOR,FA0023039,LUCKY DONUTS,PR0038158,LUCKY DONUTS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,14700 S WESTERN AVE STE # 106,GARDENA,CA,90249,1,ROUTINE INSPECTION,93,A,DAAG0SNGQ,EE0000062,nan


# Preprocessing

## Inspection Dataset

Converting the activty date into datetime format.

In [4]:
Inspections_df['ACTIVITY DATE'] = pd.to_datetime(Inspections_df['ACTIVITY DATE'], format='%m/%d/%Y')
Inspections_df.head().sort_values('ACTIVITY DATE', ascending=False)

,ACTIVITY DATE,OWNER ID,OWNER NAME,FACILITY ID,FACILITY NAME,RECORD ID,PROGRAM NAME,PROGRAM STATUS,PROGRAM ELEMENT (PE),PE DESCRIPTION,...,FACILITY CITY,FACILITY STATE,FACILITY ZIP,SERVICE CODE,SERVICE DESCRIPTION,SCORE,GRADE,SERIAL NUMBER,EMPLOYEE ID,Location
4,2017-06-15,OW0028751,PO SOKHOR,FA0023039,LUCKY DONUTS,PR0038158,LUCKY DONUTS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,GARDENA,CA,90249,1,ROUTINE INSPECTION,93,A,DAAG0SNGQ,EE0000062,NaN
3,2017-06-12,OW0100197,"SBARRO, LLC",FA0032188,SBARRO,PR0018712,SBARRO,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,LOS ANGELES,CA,90008,1,ROUTINE INSPECTION,90,A,DAMPB9559,EE0000176,NaN
2,2017-06-07,OW0007353,"CHEN, CHAO SHU",FA0070050,SAN HING,PR0004325,SAN HING,INACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,...,LOS ANGELES,CA,90034,1,ROUTINE INSPECTION,96,A,DAJMZ6U86,EE0000911,POINT (-118.422231 34.01925)
1,2017-06-05,OW0025119,MORNING SUN INC.,FA0008772,CHINA GATE,PR0029024,CHINA GATE,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,...,LOS ANGELES,CA,90059,1,ROUTINE INSPECTION,94,A,DADMCPXSY,EE0000927,NaN
0,2017-06-02,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,...,MONTEBELLO,CA,90640,1,ROUTINE INSPECTION,91,A,DAXE7ACTS,EE0000773,NaN


Moreover, note that

In [5]:
Inspections_df['FACILITY ZIP'].unique()

array(['90640', '90059', '90034', ..., '90029-1070', '93543-3116',
       '91324-1307'], dtype=object)

We would like to keep only the first five digits of each zip code (right before the dashed line). Thus,

In [6]:
Inspections_df['FACILITY ZIP'] = Inspections_df['FACILITY ZIP'].apply(lambda x: x[:5])

## Creating a new Dataset

Moreover, note that the two databases only have a mutual column

In [7]:
list(Inspections_df.columns & Violations_df.columns)

['SERIAL NUMBER']

In [8]:
Data=pd.merge(Inspections_df, Violations_df, on='SERIAL NUMBER', how='right')
Data.head().style.hide_index()

ACTIVITY DATE,OWNER ID,OWNER NAME,FACILITY ID,FACILITY NAME,RECORD ID,PROGRAM NAME,PROGRAM STATUS,PROGRAM ELEMENT (PE),PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY STATE,FACILITY ZIP,SERVICE CODE,SERVICE DESCRIPTION,SCORE,GRADE,SERIAL NUMBER,EMPLOYEE ID,Location,VIOLATION STATUS,VIOLATION CODE,VIOLATION DESCRIPTION,POINTS
2017-06-02 00:00:00,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632.000000,RESTAURANT (0-30) SEATS HIGH RISK,1111 W WASHINGTON BLVD STE H,MONTEBELLO,CA,90640,1.000000,ROUTINE INSPECTION,91.000000,A,DAXE7ACTS,EE0000773,nan,OUT OF COMPLIANCE,F036,"# 36. Equipment, utensils and linens: storage and use",1.000000
2017-06-02 00:00:00,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632.000000,RESTAURANT (0-30) SEATS HIGH RISK,1111 W WASHINGTON BLVD STE H,MONTEBELLO,CA,90640,1.000000,ROUTINE INSPECTION,91.000000,A,DAXE7ACTS,EE0000773,nan,OUT OF COMPLIANCE,F004,"# 04. Proper eating, drinking, or tobacco use",2.000000
2017-06-02 00:00:00,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632.000000,RESTAURANT (0-30) SEATS HIGH RISK,1111 W WASHINGTON BLVD STE H,MONTEBELLO,CA,90640,1.000000,ROUTINE INSPECTION,91.000000,A,DAXE7ACTS,EE0000773,nan,OUT OF COMPLIANCE,F023,"# 23. No rodents, insects, birds, or animals",2.000000
2017-06-02 00:00:00,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632.000000,RESTAURANT (0-30) SEATS HIGH RISK,1111 W WASHINGTON BLVD STE H,MONTEBELLO,CA,90640,1.000000,ROUTINE INSPECTION,91.000000,A,DAXE7ACTS,EE0000773,nan,OUT OF COMPLIANCE,F005,# 05. Hands clean and properly washed; gloves used properly,4.000000
2017-06-05 00:00:00,OW0025119,MORNING SUN INC.,FA0008772,CHINA GATE,PR0029024,CHINA GATE,ACTIVE,1632.000000,RESTAURANT (0-30) SEATS HIGH RISK,11814 S WILMINGTON AVE,LOS ANGELES,CA,90059,1.000000,ROUTINE INSPECTION,94.000000,A,DADMCPXSY,EE0000927,nan,OUT OF COMPLIANCE,F039,# 39. Wiping cloths: properly used and stored,1.000000


Moreover, note that

In [9]:
Data['PE DESCRIPTION'].unique()

array(['RESTAURANT (0-30) SEATS HIGH RISK',
       'RESTAURANT (0-30) SEATS MODERATE RISK',
       'RESTAURANT (31-60) SEATS HIGH RISK',
       'RESTAURANT (151 + ) SEATS HIGH RISK',
       'RESTAURANT (151 + ) SEATS MODERATE RISK',
       'RESTAURANT (61-150) SEATS MODERATE RISK',
       'RESTAURANT (0-30) SEATS LOW RISK',
       'RESTAURANT (61-150) SEATS HIGH RISK',
       'RESTAURANT (31-60) SEATS MODERATE RISK',
       'FOOD MKT RETAIL (2,000+ SF) LOW RISK',
       'FOOD MKT RETAIL (1-1,999 SF) LOW RISK',
       'RESTAURANT (61-150) SEATS LOW RISK',
       'FOOD MKT RETAIL (2,000+ SF) MODERATE RISK',
       'FOOD MKT RETAIL (1-1,999 SF) HIGH RISK',
       'FOOD MKT RETAIL (1-1,999 SF) MODERATE RISK',
       'FOOD MKT RETAIL (2,000+ SF) HIGH RISK',
       'RESTAURANT (151 + ) SEATS LOW RISK',
       'RESTAURANT (31-60) SEATS LOW RISK', 'CATERER (0-999 SQ. FT.)',
       'INTERIM HOUSING FF (1-1,999) SQ. FT.',
       'FOOD PROCESSING WHOLESALE (2,000-5,999 SF)',
       'INTERIM HOUSI

This column can be split into three columns.

In [10]:
temp = Data['PE DESCRIPTION'].str.split("(", n = 2, expand = True)
temp1 = temp.loc[:,1].str.split(")", n = 2, expand = True)

In [11]:
Data['PE_Type']=temp.loc[:,0]
del temp

As for types, we have the following types:

In [12]:
Data['PE_Type'].unique().tolist()

['RESTAURANT ',
 'FOOD MKT RETAIL ',
 'CATERER ',
 'INTERIM HOUSING FF ',
 'FOOD PROCESSING WHOLESALE ',
 'LIC HTH CARE FOOD FAC ',
 'SWAP MEET PREPACKAGED FOOD STAND',
 'PRIVATE SCHOOL CAFETERIA',
 'FOOD VEHICLE COMMISSARY ',
 'FOOD MARKET WHOLESALE',
 'FOOD WAREHOUSE ',
 nan]

In [13]:
Map = {'RESTAURANT ':'Restaurant',
       'FOOD MKT RETAIL ':'Food Market Retail',
       'INTERIM HOUSING FF ':'Interim Housing FF',
       'LIC HTH CARE FOOD FAC ':'LIC HTH Care Food FAC',
       'CATERER ':'Caterer',
       'SWAP MEET PREPACKAGED FOOD STAND':'Swap Meet Prepackaged Food Stand',
       'PRIVATE SCHOOL CAFETERIA':'Private School Cafeteria',
       'FOOD PROCESSING WHOLESALE ':'Food Processing Wholesale',
       'FOOD VEHICLE COMMISSARY ':'Food Vehicle Commissary',
       'FOOD WAREHOUSE ':'Food Warehouse',
       'FOOD MARKET WHOLESALE':'Food Market Wholesale'} 
Data['PE_Type'] = Data['PE_Type'].map(Map)

In [14]:
Data['PE_Seats']=temp1.loc[:,0]
Data['PE_Risk']=temp1.loc[:,1]
del temp1

Risk level:

In [15]:
Data['PE_Risk'].unique()

array([' SEATS HIGH RISK', ' SEATS MODERATE RISK', ' SEATS LOW RISK',
       ' LOW RISK', ' MODERATE RISK', ' HIGH RISK', '', ' SQ. FT.',
       'HIGH RISK', None, ' MOD RISK', nan], dtype=object)

In [16]:
Map = {' SEATS MODERATE RISK':'Moderate Risk',
       ' SEATS HIGH RISK':'High Risk',
       ' LOW RISK':'Low Risk',
       ' SEATS LOW RISK':'Low Risk',
       ' HIGH RISK':'High Risk',
       ' MODERATE RISK':'Moderate Risk', 
       ' SQ. FT.':None,
       'HIGH RISK':'High Risk',
       '':None,
       None:None,
       ' MOD RISK':'Moderate Risk'}
Data['PE_Risk'] = Data['PE_Risk'].map(Map)

In [17]:
Data['PE_Seats'].unique()

array(['0-30', '31-60', '151 + ', '61-150', '2,000+ SF', '1-1,999 SF',
       '0-999 SQ. FT.', '1-1,999', '2,000-5,999 SF', '2,000-4,999',
       '1,000-1,999 SQ. FT.', '4000-9999 SF', None, '1-1,999 SQ. FT.',
       '2000-3999 SF', '0-10', '6,000 + SQ. FT.', '0-1999 SF',
       '10,000+ SF', '1-4,999', nan], dtype=object)

In [18]:
Map = {'0-30':'0-30 SQ. FT.', '31-60':'31-60 SQ. FT.', '61-150 SQ. FT.':'61-150 SQ. FT.',
       '151 + ':'151+ SQ. FT.', '1-1,999 SF':'1-1999 SQ. FT.', '2,000+ SF':'2000+ SQ. FT.',
       '2,000-4,999':'2000-4999 SQ. FT.', '10,000+ SF':'10000+ SQ. FT.', '1-1,999':'1-1999 SQ. FT.',
       '4000-9999 SF':'4000-9999 SQ. FT.','1,000-1,999 SQ. FT.':'1000-1999 SQ. FT.',np.nan:'Other',
       None:'Other', '2,000-5,999 SF':'2000-5999 SQ. FT.', '0-999 SQ. FT.':'0-999 SQ. FT.',
       '1-1,999 SQ. FT.':'1-1999 SQ. FT.','2000-3999 SF':'2000-3999 SQ. FT.', '0-1999 SF':'0-1999 SQ. FT.',
       '0-10':'0-10 SQ. FT.','6,000 + SQ. FT.':'6000+ SQ. FT.', '1-4,999':'1-4999 SQ. FT.'}
Data['PE_Seats'] = Data['PE_Seats'].map(Map)

Droping **PE DESCRIPTION** column.

In [19]:
Data = Data.drop(columns='PE DESCRIPTION')

In [20]:
Data.head()[['PE_Type','PE_Seats','PE_Risk']].style.hide_index()

PE_Type,PE_Seats,PE_Risk
Restaurant,0-30 SQ. FT.,High Risk
Restaurant,0-30 SQ. FT.,High Risk
Restaurant,0-30 SQ. FT.,High Risk
Restaurant,0-30 SQ. FT.,High Risk
Restaurant,0-30 SQ. FT.,High Risk


### Reindexing

In [21]:
mylist = Data.columns.tolist()
mylist = [x.title() for x in mylist]
mylist = [x.replace('Id','ID') for x in mylist]
mylist = [x.replace('Pe','PE') for x in mylist]
mylist = [x.replace('  ',' ') for x in mylist]
temp = pd.DataFrame(Data.values, columns=mylist)
Data=temp.copy()
del temp, mylist

In [22]:
Data.head().style.hide_index()

Activity Date,Owner ID,Owner Name,Facility ID,Facility Name,Record ID,Program Name,Program Status,Program Element (PE),Facility Address,Facility City,Facility State,Facility Zip,Service Code,Service Description,Score,Grade,Serial Number,Employee ID,Location,Violation Status,Violation Code,Violation Description,Points,PE_Type,PE_Seats,PE_Risk
2017-06-02 00:00:00,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632.000000,1111 W WASHINGTON BLVD STE H,MONTEBELLO,CA,90640,1.000000,ROUTINE INSPECTION,91.000000,A,DAXE7ACTS,EE0000773,nan,OUT OF COMPLIANCE,F036,"# 36. Equipment, utensils and linens: storage and use",1.000000,Restaurant,0-30 SQ. FT.,High Risk
2017-06-02 00:00:00,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632.000000,1111 W WASHINGTON BLVD STE H,MONTEBELLO,CA,90640,1.000000,ROUTINE INSPECTION,91.000000,A,DAXE7ACTS,EE0000773,nan,OUT OF COMPLIANCE,F004,"# 04. Proper eating, drinking, or tobacco use",2.000000,Restaurant,0-30 SQ. FT.,High Risk
2017-06-02 00:00:00,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632.000000,1111 W WASHINGTON BLVD STE H,MONTEBELLO,CA,90640,1.000000,ROUTINE INSPECTION,91.000000,A,DAXE7ACTS,EE0000773,nan,OUT OF COMPLIANCE,F023,"# 23. No rodents, insects, birds, or animals",2.000000,Restaurant,0-30 SQ. FT.,High Risk
2017-06-02 00:00:00,OW0030652,"ROMERO, JUAN",FA0021745,EL ZIPOTE PUPUSERIA,PR0047587,EL ZIPOTE PUPUSERIA,ACTIVE,1632.000000,1111 W WASHINGTON BLVD STE H,MONTEBELLO,CA,90640,1.000000,ROUTINE INSPECTION,91.000000,A,DAXE7ACTS,EE0000773,nan,OUT OF COMPLIANCE,F005,# 05. Hands clean and properly washed; gloves used properly,4.000000,Restaurant,0-30 SQ. FT.,High Risk
2017-06-05 00:00:00,OW0025119,MORNING SUN INC.,FA0008772,CHINA GATE,PR0029024,CHINA GATE,ACTIVE,1632.000000,11814 S WILMINGTON AVE,LOS ANGELES,CA,90059,1.000000,ROUTINE INSPECTION,94.000000,A,DADMCPXSY,EE0000927,nan,OUT OF COMPLIANCE,F039,# 39. Wiping cloths: properly used and stored,1.000000,Restaurant,0-30 SQ. FT.,High Risk


In [23]:
def Data_info(Inp):
    Out = pd.DataFrame(Inp.dtypes,columns=['Data Type']).sort_values(by=['Data Type'])
    Out = Out.join(pd.DataFrame(Inp.isnull().sum(), columns=['Number of NaN Values']), how='outer')
    Out['Percentage'] = np.round(100*(Out['Number of NaN Values']/Inp.shape[0]),2)
    return Out

In [24]:
Data_info(Data)

,Data Type,Number of NaN Values,Percentage
Activity Date,datetime64[ns],62,0.01
Employee ID,object,62,0.01
Facility Address,object,62,0.01
Facility City,object,62,0.01
Facility ID,object,62,0.01
Facility Name,object,62,0.01
Facility State,object,62,0.01
Facility Zip,object,62,0.01
Grade,object,1484,0.15
Location,object,32905,3.35


### Saving

In [25]:
Violations_df.to_csv('Data/LA/Clean_Violations_df.csv',index=False)
Inspections_df.to_csv('Data/LA/Clean_Inspections_df.csv',index=False)
Data.to_csv('Data/LA/Data_Inspections_Violations_df.csv',index=False)

## Creating a Dictionary

In [26]:
Violation_Dictionary = dict.fromkeys(Data['Violation Code'].unique().tolist()) 
temp = Data['Violation Description'].unique().tolist()
for i in Violation_Dictionary:
    for j in temp:
        # Comparing the two lists
        if i[-2:] == j[2:4]:
            Violation_Dictionary[i] = j
del temp

Adding a few more codes:

In [27]:
Violation_Dictionary['F001'] = '# 01a. Demonstration of knowledge'
Violation_Dictionary['F052'] = '# 01b. Food safety certification'
Violation_Dictionary['F053'] = '# 21a. Hot Water Available'
Violation_Dictionary['F054'] = '# 52. Multiple Major Critical Violations / Increased Risk to Public Health'
Violation_Dictionary['F055'] = '# 01a. Demonstration of knowledge'
Violation_Dictionary['F057'] = '# 18. Compliance with variance, specialized process, & HACCP Plan'
Violation_Dictionary['F058'] = '# 19. Consumer advisory provided for raw or undercooked foods'

In [28]:
Violation_Dictionary_df = pd.DataFrame.from_dict(Violation_Dictionary, orient='index', columns=['Violation Description'])
Violation_Dictionary_df = Violation_Dictionary_df.sort_values('Violation Description', ascending=False)
Violation_Dictionary_df = Violation_Dictionary_df.reset_index().rename(columns={'index':'Violation Code'})
Violation_Dictionary_df.head(10)

,Violation Code,Violation Description
0,F054,# 52. Multiple Major Critical Violations / Inc...
1,W052,# 52. Multiple Major Critical Violations / Inc...
2,F051,# 51. Permit Suspension
3,W051,# 51. Permit Suspension
4,F050,# 50. Impoundment of unsanitary equipment or food
5,W050,# 50. Impoundment of unsanitary equipment or food
6,F049,# 49. Samples Collected
7,W048,# 48. Plan Review required for new or remodel ...
8,F048,# 48. Plan Review required for new or remodel ...
9,F047,# 47. Permits Available


### Saving

In [29]:
Violation_Dictionary_df.to_csv('Data/LA/Violation_Dictionary_df.csv',index=False)

***